# Modelling the risk of default for credit card applicants

This program evaluates the risk of default for credit card loan applicants. It uses logistic regression to determine model coefficients and then feeds those coefficients into a binary classification model. The binary classification model has also been built in Excel and can be viewed here: https://goo.gl/PNTDd8

In [1]:
import pandas as pd
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression

Import data for both training and testing, and optionally (if known) input the **cost per false positive** and the **cost per false negative**.

In [2]:
training_raw = pd.read_csv('training.csv', low_memory=False)
test_raw = pd.read_csv('test.csv', low_memory=False)

cost_per_false_positive = 2500
cost_per_false_negative = 5000

For ease of modelling later, add a column to the data which is simply the reverse of the default outcome:

In [3]:
training_raw['outcomes_rev'] = (training_raw.outcomes != 1).astype('int')
test_raw['outcomes_rev'] = (test_raw.outcomes != 1).astype('int')

Add a column which is the ratio of total debt to income for each application, and also a column which is the square of the given columns to see if a more complex model can provide more accurate results.

In [4]:
training_raw['debt_to_income'] = (training_raw['credit_card_debt'] + 
                              training_raw['automobile_debt']) / training_raw['income']
test_raw['debt_to_income'] = (test_raw['credit_card_debt'] + 
                              test_raw['automobile_debt']) / test_raw['income']

training_square = pd.DataFrame()
test_square = pd.DataFrame()

cols_to_square = ['age','years_employment','years_at_address','income',
                  'credit_card_debt','automobile_debt','debt_to_income']

training_square[cols_to_square] = training_raw[cols_to_square].apply(lambda x: x**2)
test_square[cols_to_square] = test_raw[cols_to_square].apply(lambda x: x**2)

training_square.rename(columns={'age':'age2','years_employment':'years_employment2',
                                'years_at_address':'years_at_address2','income':'income2',
                                'credit_card_debt':'credit_card_debt2',
                                'automobile_debt':'automobile_debt2',
                                'debt_to_income':'debt_to_income2'}, inplace=True)
test_square.rename(columns={'age':'age2','years_employment':'years_employment2',
                            'years_at_address':'years_at_address2','income':'income2',
                            'credit_card_debt':'credit_card_debt2',
                            'automobile_debt':'automobile_debt2',
                            'debt_to_income':'debt_to_income2'}, inplace=True)
training_raw = pd.concat([training_raw, training_square], axis=1)
test_raw = pd.concat([test_raw, test_square], axis=1)

Normalizing the data...

In [5]:
training_norm = pd.DataFrame()
test_norm = pd.DataFrame()

cols_to_norm = ['age','years_employment','years_at_address','income',
                'credit_card_debt','automobile_debt', 'debt_to_income', 
                'age2','years_employment2','years_at_address2','income2',
                'credit_card_debt2','automobile_debt2', 'debt_to_income2']
training_norm[cols_to_norm] = training_raw[cols_to_norm].apply(
        lambda x: (x - x.min()) / (x.max() - x.min()))
test_norm[cols_to_norm] = test_raw[cols_to_norm].apply(
        lambda x: (x - x.min()) / (x.max() - x.min()))

training_norm.rename(columns={'age':'age_norm','years_employment':'years_employment_norm',
                'years_at_address':'years_at_address_norm','income':'income_norm',
                'credit_card_debt':'credit_card_debt_norm',
                'automobile_debt':'automobile_debt_norm',
                'debt_to_income':'debt_to_income_norm', 
                'age2':'age2_norm','years_employment2':'years_employment2_norm',
                'years_at_address2':'years_at_address2_norm','income2':'income2_norm',
                'credit_card_debt2':'credit_card_debt2_norm',
                'automobile_debt2':'automobile_debt2_norm',
                'debt_to_income2':'debt_to_income2_norm'}, inplace=True)
test_norm.rename(columns={'age':'age_norm','years_employment':'years_employment_norm',
                'years_at_address':'years_at_address_norm','income':'income_norm',
                'credit_card_debt':'credit_card_debt_norm',
                'automobile_debt':'automobile_debt_norm',
                'debt_to_income':'debt_to_income_norm', 
                'age2':'age2_norm','years_employment2':'years_employment2_norm',
                'years_at_address2':'years_at_address2_norm','income2':'income2_norm',
                'credit_card_debt2':'credit_card_debt2_norm',
                'automobile_debt2':'automobile_debt2_norm',
                'debt_to_income2':'debt_to_income2_norm'}, inplace=True)

training_raw = pd.concat([training_raw, training_norm], axis=1)
test_raw = pd.concat([test_raw, test_norm], axis=1)

...and set initial intercepts to 1

In [6]:
training_raw['intercept'] = 1
test_raw['intercept'] = 1

Now, we need to define several helper functions to run the model.

First up, is a function to calculate the thresholds for the binary classification model. It simply finds the maximum and minimum scores and makes a number of evenly spaced out bins.

In [7]:
def calc_thresholds(df, num_thresholds):
    '''
    helper function to calculate the threshold values
    '''
    threshold_step = (df['scores'].max() - df['scores'].min()) / (num_thresholds - 1)
    threshold = df['scores'].min()
    thresholds = [threshold]
    while threshold < df['scores'].max():
        threshold += threshold_step
        thresholds.append(threshold)
    return list(reversed(thresholds))

Next, we need a function to calculate the scores matrix

In [8]:
def scores(df, coefficients):
    '''
    helper function to multiply a DataFrame by coefficients
    '''
    scores = (df * coefficients).sum(axis=1)
    scores = pd.DataFrame(scores)
    scores.columns = ['scores']
    
    return scores

The next three functions are necessary to calculate the **Area Under The ROC Curve**

In [9]:
def calc_condition_incidence(df):
    total_outcomes = len(df)
    total_outcomes_1 = len(df[df['outcomes'] == 1])
    total_outcomes_0 = total_outcomes - total_outcomes_1
    
    condition_incidence = total_outcomes_1 / total_outcomes
    
    return (total_outcomes, total_outcomes_1, total_outcomes_0, condition_incidence)


def true_positive_analysis(df, thresholds):
    '''
    helper function to calculate the true positives at each threshold
    '''
    df = df.copy()
    for threshold in thresholds:
        df[str(threshold)] = df[['scores','outcomes']].apply(lambda x: 
            x['outcomes'] if x['scores']>threshold else 0, axis=1)

    return df
    
    
def false_positive_analysis(df, thresholds):
    '''
    helper function to calculate the false positives at each threshold
    '''
    df = df.copy()
    for threshold in thresholds:
        df[str(threshold)] = df[['scores','outcomes_rev']].apply(lambda x: 
            x['outcomes_rev'] if x['scores']>threshold else 0, axis=1)

    return df

Next up is a function to create the **Area Under The Curve matrix**

In [10]:
def area_under_the_curve(df_positive, df_negative, thresholds, total_outcomes, total_outcomes_1, total_outcomes_0):
    '''
    helper function to calculate the area under the receiver operating 
    characteristic (ROC) curve and are-under-the-curve
    '''
    
    autc = pd.DataFrame(index=('TPs at threshold','TP rate','FPs at threshold',
             'FP rate','Area of Rectangle','FNs at threshold','Cost Per Event'))
    tp_rate_prev = 0
    fp_rate_prev = 0

    for threshold in thresholds:
        threshold = str(threshold)
        true_positives = df_positive[threshold].sum(axis=0)
        autc.set_value('TPs at threshold', threshold, true_positives)
        tp_rate = true_positives / total_outcomes_1
        autc.set_value('TP rate', threshold, tp_rate)
        false_positives = df_negative[threshold].sum(axis=0)
        autc.set_value('FPs at threshold', threshold, false_positives)
        fp_rate = false_positives / total_outcomes_0
        autc.set_value('FP rate', threshold, fp_rate)
        if threshold != thresholds[0]:
            area_of_rectangle = ((tp_rate + tp_rate_prev) * 
                                (fp_rate - fp_rate_prev)) / 2
        tp_rate_prev = tp_rate
        fp_rate_prev = fp_rate
        autc.set_value('Area of Rectangle', threshold, area_of_rectangle)
        autc.set_value('FNs at threshold', threshold, total_outcomes_1 - true_positives)
        total_area = autc.sum(axis=1)['Area of Rectangle']
        if total_area < .5:
            total_area = 1 - total_area
        
    return autc, total_area

Lastly, a function to calculate the **minimum-cost-per-event**, which is only called if the **cost-per-false-positive** and **cost-per-false-negative** are known.

In [11]:
def minimum_cost_per_event(autc, cost_per_false_positive, cost_per_false_negative):
    '''
    helper function to calculate the minimum cost per event for a given matrix
    '''

    total_outcomes = (len(autc.columns))
    autc = autc.T
    autc['Cost Per Event'] = (cost_per_false_negative * autc['FNs at threshold'] + 
        cost_per_false_positive * autc['FPs at threshold']) / total_outcomes
    min_cost_per_event = autc['Cost Per Event'].min(axis=0)
    min_threshold = autc['Cost Per Event'].idxmin(axis=0)
    TP_rate = autc.get_value(min_threshold, 'TP rate')
    FP_rate = autc.get_value(min_threshold, 'FP rate')
    
    return min_cost_per_event, min_threshold, autc, TP_rate, FP_rate

We now have all the necessary helper function defined, and so now we build a function to run the full model. When called, this function takes the specified features and runs logistic regression to calculate correlations between the features. It then uses the previously defined helper functions to run through a binary classification model. If the **cost per false positive** and **cost per false negative** are not known, the best way to evaluate the model's performance is by maximizing the total **area under the curve**. However, if those values are known, then the function will determine the threshold which corresponds to the **minimum cost per event**. In both cases, the function return all values for both the training data and the test data.

In [12]:
def run_model(features, cost_per_false_positive=None, cost_per_false_negative=None):
    training = training_raw.copy()
    test = test_raw.copy()
    
    y_values = 'outcomes ~ ' + ' + '.join(features)

    features.insert(0, 'intercept')
    
    y, X = dmatrices(y_values,training,return_type='dataframe')
    
    model = LogisticRegression(fit_intercept = False).fit(X,y.values.ravel())
    
    model_scores = scores(training[features],model.coef_[0])

    model_scores['outcomes'] = training['outcomes']
    model_scores['outcomes_rev'] = training['outcomes_rev']
    
    total_outcomes, total_outcomes_1, total_outcomes_0, condition_incidence = calc_condition_incidence(training)
    
    model_thresholds = calc_thresholds(model_scores, 200)
    
    TP_model_scores = true_positive_analysis(model_scores, model_thresholds)
    FP_model_scores = false_positive_analysis(model_scores, model_thresholds)
    
    autc_matrix, autc = area_under_the_curve(TP_model_scores, FP_model_scores,model_thresholds, 
                            total_outcomes, total_outcomes_1, total_outcomes_0)
    
    min_cost_per_event = None
    
    if cost_per_false_positive != None and cost_per_false_negative != None:
        min_cost_per_event, min_threshold, min_cost_matrix, TP_rate, FP_rate = minimum_cost_per_event(autc_matrix, 
                              cost_per_false_positive, cost_per_false_negative)
        
        prob_TP = TP_rate * condition_incidence
        prob_FP = FP_rate * condition_incidence
        classification_incidence = prob_TP + prob_FP
    else:
        min_cost_per_event = None
        min_threshold = None
        TP_rate = None
        FP_rate = None
        prob_TP = None
        prob_FP = None
        classification_incidence = None
    
    test_scores = scores(test[features],model.coef_[0])
    test_scores['outcomes'] = test['outcomes']
    test_scores['outcomes_rev'] = test['outcomes_rev']
    test_outcomes, test_outcomes_1, test_outcomes_0, test_condition_incidence = calc_condition_incidence(training)
    
    TP_test_scores = true_positive_analysis(test_scores, model_thresholds)
    FP_test_scores = false_positive_analysis(test_scores, model_thresholds)
    test_autc_matrix, test_autc = area_under_the_curve(TP_test_scores, FP_test_scores, model_thresholds, 
                            test_outcomes, test_outcomes_1, test_outcomes_0)
    min_cost_per_event_test = None
    if cost_per_false_positive != None and cost_per_false_negative != None:
        a,b,cost_per_event_matrix, test_TP_rate, test_FP_rate = minimum_cost_per_event(test_autc_matrix, cost_per_false_positive, cost_per_false_negative)
        min_cost_per_event_test = cost_per_event_matrix.get_value(min_threshold, 'Cost Per Event')
        test_prob_TP = test_TP_rate * test_condition_incidence
        test_prob_FP = test_FP_rate * test_condition_incidence
        test_classification_incidence = test_prob_TP + test_prob_FP
    else:
        cost_per_event_matrix = None
        test_TP_rate = None
        test_FP_rate = None
        min_cost_per_event_test = None
        test_prob_TP = None
        test_prob_FP = None
        test_classification_incidence = None
    
    return (features, model.coef_[0], autc, test_autc, min_threshold, min_cost_per_event, 
            min_cost_per_event_test, condition_incidence, test_condition_incidence, 
            prob_TP, test_prob_TP, classification_incidence, test_classification_incidence)

For the first model to try, let's use a basic feature list with all features available and in a linear format, then let's call the *run_model* function.

In [13]:
features1 = ['age','years_employment','years_at_address','income',
             'credit_card_debt','automobile_debt']

In [14]:
model1 = run_model(features1, cost_per_false_positive, cost_per_false_negative)

In order to view the results, it is helpful to define another helper function which will format the information

In [15]:
def print_results(model):
    print ('|=================================================================|')
    print ('|                   Feature     Coefficient                       |')
    print ('|                   -------     -----------                       |')
    for idx in range(len(model[0])):
        print ('|', '{:>25}'.format(model[0][idx]), ' : ', '{:<25}'.format(
                model[1][idx]), '        |')
    print ('|                                                                 |')
    print ('|                                   Training       Test           |')
    print ('|         Area Under the ROC Curve:', '{:<10}'.format(
            round(float(model[2]), 5)), '   ', '{:<10}'.format(
                    round(float(model[3]), 5)), '    |')
    if model[4] != None:
        print ('|           Minimum Cost Per Event:', '{:<10}'.format(
                round(float(model[5]), 5)), '   ', '{:<10}'.format
            (round(float(model[6]), 5)), '    |')
        print ('|                     at threshold:', '{:<20}'.format
               (round(float(model[4]), 10)), '         |')
        print ('|              Condition Incidence:', '{:<10}'.format(
                round(float(model[7]), 5)), '   ', '{:<10}'.format(
                        round(float(model[8]), 5)), '    |')
        print ('|    Probability of True Positives:', '{:<10}'.format(
                round(float(model[9]), 5)), '   ', '{:<10}'.format(
                        round(float(model[10]), 5)), '    |')
        print ('|  Test (Classification) Indidence:', '{:<10}'.format
               (round(float(model[11]), 5)), '   ', '{:<10}'.format(
                       round(float(model[12]), 5)), '    |')
    print ('|=================================================================|')
    print ('')
    
    return

Let's view the results from this first model:

In [16]:
print_results(model1)

|=================================================================|
|                   Feature     Coefficient                       |
|                   -------     -----------                       |
|                 intercept  :  0.002751246713909548              |
|                       age  :  -0.014182070323539155             |
|          years_employment  :  -0.23551163634047798              |
|          years_at_address  :  0.01238765406651646               |
|                    income  :  -1.8770999393600802e-05           |
|          credit_card_debt  :  -0.0004000758764461891            |
|           automobile_debt  :  -7.202038663058489e-05            |
|                                                                 |
|                                   Training       Test           |
|         Area Under the ROC Curve: 0.839          0.83207        |
|           Minimum Cost Per Event: 637.5          700.0          |
|                     at threshold: -0.726674750

Because the difference **Area Under the ROC Curve** for the training data and the test data, we can tell that this model is quite robust. If we didn't know the **cost per false positive** and **cost per false negative**, we'd stop here and know our model is pretty good. But with that information included, we can see that **Minimum Cost Per Event** for both training and test data are also very close. This also tells us our threshold: -0.7266747506.

We now know that for a bank to maximize profits, they should put each application to a test of:

      0.002751246713909548
    + age * -0.014182070323539155
    + years_employment * -0.23551163634047798
    + years_at_address * 0.01238765406651646
    + income * -1.8770999393600802e-05
    + credit_card_debt * -0.000400075876446189
    + automobile_debt * -7.202038663058489e-05

If this value is greater than the threshold, **-0.7266747506**, the bank should reject the credit card application. In the data given, there is a 25% default rate on 600 applications, with a cost per false negative of \$5000. If the bank were to simply accept all applications, the defaulted credit card accounts would cost the bank

    25% * $5000 * 600
    = $750,000
    
which results in a cost per application event of 750,000 / 600 = **\$1250**. This logistic regression/binary classification model has a cost per event of only \$700 on the test set, which indicates that this model would save the bank 1250 - 700 = **$550** per application, ***if we assume that these 600 applications are representative of all future applications***.

Now let's compare this model with a few others:

In [17]:
print ('===========Model 1===========')
print_results(model1)

features2 = ['age','years_employment','years_at_address','debt_to_income']
model2 = run_model(features2, cost_per_false_positive, cost_per_false_negative)
print ('===========Model 2===========')
print_results(model2)

features3 = ['age','years_employment','years_at_address','income',
             'credit_card_debt','automobile_debt', 'debt_to_income']
model3 = run_model(features3, cost_per_false_positive, cost_per_false_negative)
print ('===========Model 3===========')
print_results(model3)

features4 = ['age','age2','years_employment','years_employment2',
             'years_at_address','years_at_address2','income','income2',
             'credit_card_debt','credit_card_debt2','automobile_debt',
             'automobile_debt2','debt_to_income','debt_to_income2']
model4 = run_model(features4, cost_per_false_positive, cost_per_false_negative)
print ('===========Model 4===========')
print_results(model4)

features5 = ['age','age2','years_employment','years_employment2',
             'years_at_address','years_at_address2','debt_to_income','debt_to_income2']
model5 = run_model(features5, cost_per_false_positive, cost_per_false_negative)
print ('===========Model 5===========')
print_results(model5)

features6 = ['age_norm','years_employment_norm','years_at_address_norm','income_norm',
             'credit_card_debt_norm','automobile_debt_norm']
model6 = run_model(features6, cost_per_false_positive, cost_per_false_negative)
print ('===========Model 6===========')
print_results(model6)

features7 = ['age_norm','years_employment_norm','years_at_address_norm','debt_to_income']
model7 = run_model(features7, cost_per_false_positive, cost_per_false_negative)
print ('===========Model 7===========')
print_results(model7)

features8 = ['age_norm','years_employment_norm','years_at_address_norm','debt_to_income_norm']
model8 = run_model(features8, cost_per_false_positive, cost_per_false_negative)
print ('===========Model 8===========')
print_results(model8)

===========Model 1===========
|=================================================================|
|                   Feature     Coefficient                       |
|                   -------     -----------                       |
|                 intercept  :  0.002751246713909548              |
|                       age  :  -0.014182070323539155             |
|          years_employment  :  -0.23551163634047798              |
|          years_at_address  :  0.01238765406651646               |
|                    income  :  -1.8770999393600802e-05           |
|          credit_card_debt  :  -0.0004000758764461891            |
|           automobile_debt  :  -7.202038663058489e-05            |
|                                                                 |
|                                   Training       Test           |
|         Area Under the ROC Curve: 0.839          0.83207        |
|           Minimum Cost Per Event: 637.5          700.0          |
|                 

By looking at the **area under the curve** of each model, and the values of and differences between the **minimum cost per events**, we can see that Model 1, the simpliest model, is actually very nearly the best. The only model to edge it out is Model 3, which adds the additional feature of **debt to income ratio**. Because this model is slightly more complex though and the gain is very minimal, I am tempted to use Model 1. There is a risk of overfitting when model complexity increases so I do not think the minimal gains are worth the additional complexity. Therefore, Model 1 is the bank's best bet.



*IF*
**0.00275 - [age] \* 0.0142 - [years_employment] \* 0.236 + [years_at_address] \* 0.0124 - [income] \* 0.0000188 - [credit_card_debt] \* -0.000400 - [automobile_debt] \* 0.0000720**
*is greater than* **-0.7266747506**,
***reject*** the applicant